# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,zakamensk,50.3741,103.2863,244.91,99,59,0.49,RU,1702248881
1,1,imbituba,-28.2400,-48.6703,292.97,96,100,4.87,BR,1702248881
2,2,iqaluit,63.7506,-68.5145,268.00,79,100,5.14,CA,1702248882
3,3,petropavlovsk-kamchatsky,53.0452,158.6483,274.48,95,100,6.71,RU,1702248883
4,4,adamstown,-25.0660,-130.1015,294.23,58,0,4.33,PN,1702248823


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points('Lng','Lat',geo = True,tiles = 'OSM',
                                      color = 'City',
                                      size = 'Humidity')

# Display the map
city_map

c:\Users\EvanMucciolo\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Convert Max Temp from Kelvin to Celsius
city_data_df['Max Temp'] = city_data_df['Max Temp'].apply(lambda x: x - 273.15)

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_ideal_df = city_data_df.loc[(city_data_df['Max Temp']> 21)&(city_data_df['Max Temp']< 27)&(city_data_df['Wind Speed']< 4.5)
                                       &(city_data_df['Cloudiness'] < 30)
                                      ,:]

# Drop any rows with null values
city_data_ideal_df = city_data_ideal_df.dropna()

# Display sample data
city_data_ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,adamstown,-25.0660,-130.1015,21.08,58,0,4.33,PN,1702248823
5,5,maceio,-9.6658,-35.7353,25.69,83,0,3.09,BR,1702248884
120,120,mercedes,-34.6515,-59.4307,25.48,50,0,4.02,AR,1702248962
124,124,georgetown,5.4112,100.3354,26.96,92,20,3.09,MY,1702248964
131,131,belmonte,-15.8631,-38.8828,25.27,82,8,3.51,BR,1702248969


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_ideal_df[['City', 'Country','Lat','Lng','Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

C:\Users\EvanMucciolo\AppData\Local\Temp\ipykernel_28060\1979163482.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name'] = ''


,City,Country,Lat,Lng,Humidity,Hotel Name
4,adamstown,PN,-25.0660,-130.1015,58,
5,maceio,BR,-9.6658,-35.7353,83,
120,mercedes,AR,-34.6515,-59.4307,50,
124,georgetown,MY,5.4112,100.3354,92,
131,belmonte,BR,-15.8631,-38.8828,82,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [13]:
# Set parameters to search for a hotel
radius = 10000
categories = 'accommodation.hotel'


params = {
    "categories":categories,
    "apiKey":geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row[2]
    lon = row[3]
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search


C:\Users\EvanMucciolo\AppData\Local\Temp\ipykernel_28060\3703366581.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lat = row[2]
C:\Users\EvanMucciolo\AppData\Local\Temp\ipykernel_28060\3703366581.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lon = row[3]


adamstown - nearest hotel: No hotel found
maceio - nearest hotel: Maceió Hostel e Pousada
mercedes - nearest hotel: Gran Hotel Mercedes
georgetown - nearest hotel: Page 63 hostel
belmonte - nearest hotel: No hotel found
ramona - nearest hotel: No hotel found
al ghayzah - nearest hotel: فندق تاج العرب
beinamar - nearest hotel: No hotel found
alderetes - nearest hotel: Sheraton Tucuman Hotel
lihue - nearest hotel: Kauai Palms
dwarka - nearest hotel: The Dwarika Hotel
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
porbandar - nearest hotel: Toran Tourist Bungalow
pitoa - nearest hotel: No hotel found
myeik - nearest hotel: Green Eyes Hotel
salalah - nearest hotel: Muscat International Hotel
maturin - nearest hotel: Hotel Rossi
berbera - nearest hotel: Al Madina Hotel
harper - nearest hotel: No hotel found
hengchun - nearest hotel: 海的顏色精品旅館
brinkmann - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
4,adamstown,PN,-25.0660,-130.1015,58,No hotel found
5,maceio,BR,-9.6658,-35.7353,83,Maceió Hostel e Pousada
120,mercedes,AR,-34.6515,-59.4307,50,Gran Hotel Mercedes
124,georgetown,MY,5.4112,100.3354,92,Page 63 hostel
131,belmonte,BR,-15.8631,-38.8828,82,No hotel found
222,ramona,US,33.0417,-116.8681,21,No hotel found
224,al ghayzah,YE,16.2079,52.1760,63,فندق تاج العرب
226,beinamar,TD,8.6698,15.3813,23,No hotel found
245,alderetes,AR,-26.8167,-65.1333,65,Sheraton Tucuman Hotel
246,lihue,US,21.9789,-159.3672,76,Kauai Palms


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
#%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points('Lng','Lat',geo = True,tiles = 'OSM',
                                      color = 'City',
                                      size = 'Humidity',
                                      hover_cols=['Hotel Name','Country'])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)